![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb)

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
  
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline,PipelineModel

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.3.2
Spark NLP_JSL Version : 4.3.2


# Clinical NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `embeddings_clinical`.

<center><b>NER Profiling Clinical Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| jsl_ner_wip_clinical | jsl_ner_wip_greedy_clinical | jsl_ner_wip_modifier_clinical | jsl_rd_ner_wip_greedy_clinical |
| ner_abbreviation_clinical | ner_ade_binary | ner_ade_clinical | ner_anatomy |
| ner_anatomy_coarse | ner_bacterial_species | ner_biomarker | ner_biomedical_bc2gm |
| ner_bionlp | ner_cancer_genetics | ner_cellular | ner_chemd_clinical |
| ner_chemicals | ner_chemprot_clinical | ner_chexpert | ner_clinical |
| ner_clinical_large | ner_clinical_trials_abstracts | ner_covid_trials | ner_deid_augmented |
| ner_deid_enriched | ner_deid_generic_augmented | ner_deid_large | ner_deid_sd |
| ner_deid_sd_large | ner_deid_subentity_augmented | ner_deid_subentity_augmented_i2b2 | ner_deid_synthetic |
| ner_deidentify_dl | ner_diseases | ner_diseases_large | ner_drugprot_clinical |
| ner_drugs | ner_drugs_greedy | ner_drugs_large | ner_events_admission_clinical |
| ner_events_clinical | ner_genetic_variants | ner_human_phenotype_gene_clinical | ner_human_phenotype_go_clinical |
| ner_jsl | ner_jsl_enriched | ner_jsl_greedy | ner_jsl_slim |
| ner_living_species | ner_measurements_clinical | ner_medmentions_coarse | ner_nature_nero_clinical |
| ner_nihss | ner_pathogen | ner_posology | ner_posology_experimental |
| ner_posology_greedy | ner_posology_large | ner_posology_small | ner_radiology |
| ner_radiology_wip_clinical | ner_risk_factors | ner_supplement_clinical | nerdl_tumour_demo |

</center>

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [ ]:
document_assembler = DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

#NER model to detect drug in the text
ner = MedicalNerModel.pretrained('ner_posology_greedy', 'en', 'clinical/models') \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_chunk = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

chunkMapper = ChunkMapperModel.pretrained("drug_ade_mapper", "en", "clinical/models")\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("mappings")\
      .setRels(["ADE"])

pipeline = Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer, 
                                 word_embeddings,
                                 ner, 
                                 ner_chunk, 
                                 chunkMapper])

text = ["""The patient was prescribed 1000 mg fish oil and multivitamins. 
            She was discharged on zopiclone and ambrisentan"""]
            
test_data = spark.createDataFrame([text]).toDF("text")
model = pipeline.fit(test_data)
res= model.transform(test_data)
res.show()

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_greedy download started this may take some time.
[OK!]
drug_ade_mapper download started this may take some time.
[OK!]
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|            mappings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The patient was p...|[{document, 0, 12...|[{document, 0, 61...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 27, 42, ...|[{labeled_depende...|
+--------------------+--------------------+--------------------+--------------------

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.5 GB
[OK!]


In [ ]:
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''

In [ ]:
clinical_result = clinical_profiling_pipeline.annotate(text)
clinical_result.keys()

dict_keys(['ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_posology_greedy_chunks', 'ner_radiology_wip_clinical', 'ner_deidentify_dl', 'ner_jsl_slim', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_deid_synthetic', 'ner_drugs_greedy', 'ner_abbreviation_clinical_chunks', 'ner_covid_trials_chunks', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greedy', 'ner_cellular_chunks', 'ner_cancer_genetics_chunks', 'ner_biomedical_bc2gm_chunks', 'ner_jsl_greedy', 'jsl_ner_wip_modifier_clinical_chunks', 'ner_drugs_greedy_chunks', 'ner_deid_sd_large_chunks', 'ner_diseases_chunks', 'ner_diseases_large', 'ner_chemprot_clinical', 'ner_posology_large', 'nerdl_tumour_demo_chunks', 'ner_deid_subentity_augmented_chunks', 'ner_jsl_enriched_chunks', 'ner_genetic_variants_chunks', 'ner_chexpert', 'ner_bionlp_chunks', 'ner_measurements_clinical_chunks', 'ner_diseases_larg

In [ ]:
print("Clinical Text: \n", text, "\n \nResults:\n")

for i in clinical_result.keys():
  print("{} : ".format(i), clinical_result[i])

Clinical Text: 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
 
Results:

ner_ade_clinical_chunks :  ['polydipsia', 'poor appetite', 'vomiting']
ner_deid_augmented :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
ner_deid_subentity_augmented_i2b2 :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

**Lets show chunk results of NER models in a pandas dataframe.**

In [ ]:
import pandas as pd


def get_chunk_results(light_result):

  for i in light_result.keys():
    
    model_name = "_".join(i.split("_")[:-1])

    if (i == 'sentence') | (i == 'token'):
      
      continue
    
    if ("_chunks" not in i):
      
      continue

    elif len(light_result[i]) == 0:
      
      print("\n", "*"*20, model_name, "Model Results", "*"*20)
      print("No Result For This Model")


    else:

      sentences = []
      begins = []
      ends = []
      chunks = []
      entities = []
      confidences = []

      for n in (light_result[i]):

        sentences.append(n.metadata['sentence'])
        begins.append(n.begin)
        ends.append(n.end) 
        chunks.append(n.result)
        entities.append(n.metadata['entity'])
        confidences.append(n.metadata['confidence'])

        
      df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'chunks': chunks, 'entity': entities, 'confidence': confidences})

      print("\n \n", "*"*20, model_name, "Model Results", "*"*20)
      display(df)

In [ ]:
text2 = """
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007
"""

In [ ]:
clinical_full_result = clinical_profiling_pipeline.fullAnnotate(text2)[0]

In [ ]:
clinical_full_result['ner_jsl_chunks']

[Annotation(chunk, 15, 34, chest tube placement, {'entity': 'Procedure', 'sentence': '0', 'chunk': '0', 'confidence': '0.46089998'}, []),
 Annotation(chunk, 40, 60, drainage of the fluid, {'entity': 'Procedure', 'sentence': '0', 'chunk': '1', 'confidence': '0.40230003'}, []),
 Annotation(chunk, 75, 86, thoracoscopy, {'entity': 'Procedure', 'sentence': '0', 'chunk': '2', 'confidence': '0.9927'}, []),
 Annotation(chunk, 93, 106, fluid biopsies, {'entity': 'Procedure', 'sentence': '0', 'chunk': '3', 'confidence': '0.80535'}, []),
 Annotation(chunk, 146, 179, epithelioid malignant mesothelioma, {'entity': 'Oncological', 'sentence': '0', 'chunk': '4', 'confidence': '0.934'}, []),
 Annotation(chunk, 219, 224, PET CT, {'entity': 'Test', 'sentence': '1', 'chunk': '5', 'confidence': '0.946'}, []),
 Annotation(chunk, 240, 248, extensive, {'entity': 'Modifier', 'sentence': '1', 'chunk': '6', 'confidence': '0.8028'}, []),
 Annotation(chunk, 250, 255, uptake, {'entity': 'ImagingFindings', 'sentence

In [ ]:
print("Clinical Text:\n", text2)

Clinical Text:
 
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007



In [ ]:
get_chunk_results(clinical_full_result)


 
 ******************** ner_ade_clinical Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,DRUG,0.9326
1,0,40,60,drainage of the fluid,DRUG,0.66934997
2,5,600,611,chemotherapy,DRUG,0.9953
3,5,639,647,cisplatin,DRUG,0.9877
4,5,690,701,IV piggyback,DRUG,0.6635



 
 ******************** ner_posology_greedy Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,5,600,611,chemotherapy,DRUG,0.9972
1,5,639,664,cisplatin 75 mg/centimeter,DRUG,0.6113667
2,5,683,701,109 mg IV piggyback,DRUG,0.55439997
3,5,703,714,over 2 hours,DURATION,0.5100667



 
 ******************** ner_risk_factors Model Results ********************


...

,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,Procedure,0.38986668
1,0,75,86,thoracoscopy,Procedure,0.9627
2,0,93,106,fluid biopsies,Procedure,0.6287
3,0,146,179,epithelioid malignant mesothelioma,Oncological,0.77963334
4,1,219,224,PET CT,Test,0.61195004
5,1,240,268,extensive uptake in the chest,Symptom,0.42046
6,1,271,279,bilateral,Direction,0.4811
7,1,281,309,pleural pericardial effusions,Disease_Syndrome_Disorder,0.35029998
8,1,316,330,lymphadenopathy,Symptom,0.9886
9,2,334,336,She,Gender,0.9986



 
 ******************** ner_ade_binary Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,146,179,epithelioid malignant mesothelioma,ADE,0.6943
1,1,250,268,uptake in the chest,ADE,0.5401
2,1,271,309,bilateral pleural pericardial effusions,ADE,0.610125
3,1,316,330,lymphadenopathy,ADE,0.9985
4,2,347,358,acidic fluid,ADE,0.6749
5,2,361,397,pectoral and intramammary lymph nodes,ADE,0.72288
6,2,403,423,uptake in L4 with SUV,ADE,0.76716
7,3,462,472,III disease,ADE,0.54185
8,4,511,527,ejection fraction,ADE,0.72225



 
 ******************** ner_events_admission_clinical Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,TREATMENT,0.8781667
1,0,40,60,drainage of the fluid,TREATMENT,0.4451
2,0,75,86,thoracoscopy,TEST,0.9948
3,0,93,106,fluid biopsies,TEST,0.84274995
4,0,137,144,revealed,EVIDENTIAL,0.5703
5,0,146,179,epithelioid malignant mesothelioma,PROBLEM,0.87450004
6,1,204,210,stained,PROBLEM,0.3036
7,1,217,224,a PET CT,TEST,0.74953336
8,1,233,238,showed,EVIDENTIAL,0.9502
9,1,240,268,extensive uptake in the chest,PROBLEM,0.73456


**Now we will show all NER labels of tokens in the same dataframe.**

In [ ]:
import pandas as pd

def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:
    
    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)
    
  return df

In [ ]:
get_token_results(clinical_full_result)

,sentence,begin,end,token,ner_deid_augmented,ner_deid_subentity_augmented_i2b2,ner_radiology_wip_clinical,ner_deidentify_dl,ner_jsl_slim,ner_deid_synthetic,...,ner_posology_experimental,jsl_ner_wip_clinical,ner_biomedical_bc2gm,ner_jsl,ner_events_clinical,ner_supplement_clinical,ner_genetic_variants,ner_radiology,ner_posology,ner_covid_trials
0,0,1,2,At,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
1,0,4,7,this,O,O,O,O,O,O,...,O,O,O,O,B-DATE,O,O,O,O,O
2,0,9,12,time,O,O,O,O,O,O,...,O,O,O,O,I-DATE,O,O,O,O,O
3,0,13,13,",",O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
4,0,15,19,chest,O,O,B-Procedure,O,B-Procedure,O,...,O,B-Procedure,O,B-Procedure,B-TREATMENT,O,O,B-Procedure,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,5,716,717,on,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
124,5,719,727,September,B-DATE,B-DATE,O,B-DATE,B-Date_Time,B-DATE,...,O,B-Date,O,B-Date,B-DATE,O,O,O,O,B-Date
125,5,729,729,1,I-DATE,I-DATE,O,I-DATE,I-Date_Time,I-DATE,...,O,I-Date,O,I-Date,I-DATE,O,O,O,O,I-Date
126,5,730,730,",",I-DATE,I-DATE,O,I-DATE,I-Date_Time,I-DATE,...,O,I-Date,O,I-Date,I-DATE,O,O,O,O,I-Date


# BioBert NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `biobert_pubmed_base_cased`.

<center><b>NER Profiling BioBert Model List</b>

| | |
|-|-|
| ner_cellular_biobert           | ner_clinical_biobert             |
| ner_diseases_biobert           | ner_anatomy_coarse_biobert       |
| ner_events_biobert             | ner_human_phenotype_gene_biobert |
| ner_bionlp_biobert             | ner_posology_large_biobert       |
| ner_jsl_greedy_biobert         | jsl_rd_ner_wip_greedy_biobert    |
| ner_jsl_biobert                | ner_posology_biobert             |
| ner_anatomy_biobert            | jsl_ner_wip_greedy_biobert       |
| ner_jsl_enriched_biobert       | ner_chemprot_biobert             |
| ner_human_phenotype_go_biobert | ner_ade_biobert                  |
| ner_deid_biobert               | ner_risk_factors_biobert         |
| ner_deid_enriched_biobert      | ner_living_species_biobert                                |


</center>

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

biobert_profiling_pipeline = PretrainedPipeline("ner_profiling_biobert", "en", "clinical/models")

ner_profiling_biobert download started this may take some time.
Approx size to download 730.9 MB
[OK!]


In [ ]:
bio_result = biobert_profiling_pipeline.fullAnnotate(text)[0]
bio_result.keys()

dict_keys(['ner_ade_biobert', 'ner_cellular_biobert_chunks', 'ner_human_phenotype_go_biobert', 'ner_events_biobert', 'ner_diseases_biobert_chunks', 'ner_events_biobert_chunks', 'jsl_ner_wip_greedy_biobert', 'ner_bionlp_biobert_chunks', 'ner_bionlp_biobert', 'ner_jsl_greedy_biobert_chunks', 'ner_jsl_biobert_chunks', 'ner_jsl_biobert', 'ner_anatomy_biobert_chunks', 'ner_posology_large_biobert', 'ner_jsl_enriched_biobert_chunks', 'jsl_rd_ner_wip_greedy_biobert', 'ner_human_phenotype_go_biobert_chunks', 'ner_posology_biobert', 'ner_deid_biobert_chunks', 'ner_jsl_greedy_biobert', 'ner_chemprot_biobert', 'ner_deid_enriched_biobert_chunks', 'ner_jsl_enriched_biobert', 'token', 'ner_deid_biobert', 'ner_clinical_biobert_chunks', 'ner_human_phenotype_gene_biobert', 'ner_anatomy_coarse_biobert_chunks', 'ner_human_phenotype_gene_biobert_chunks', 'ner_posology_large_biobert_chunks', 'jsl_rd_ner_wip_greedy_biobert_chunks', 'ner_posology_biobert_chunks', 'ner_clinical_biobert', 'ner_diseases_biobert'

In [ ]:
print("Clinical Text: \n", text, "\n \nResults:\n")

for i in bio_result.keys():
  print("{} : ".format(i), bio_result[i])

Clinical Text: 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
 
Results:

ner_ade_biobert :  [Annotation(named_entity, 0, 0, O, {'word': 'A', 'confidence': '0.9999', 'sentence': '0'}, []), Annotation(named_entity, 2, 12, O, {'word': '28-year-old', 'confidence': '0.9997', 'sentence': '0'}, []), Annotation(named_entity, 14, 19, O, {'word': 'female', 'confidence': '0.978', 'sentence': '0'}, []), Annotation(named_entity, 21, 24, O, {'word': 'with', 'confidence': '1.0', 'sentence': '0'}, []), Annotation(named_entity, 26, 26, O, {'word': 'a', 'confidence': '0.9993', 'sentence': '0'}, []), Annotation(named_en

In [ ]:
bio_full_result = biobert_profiling_pipeline.fullAnnotate(text2)[0]

In [ ]:
bio_full_result['ner_jsl_biobert_chunks']

[Annotation(chunk, 15, 34, chest tube placement, {'entity': 'Procedure', 'sentence': '0', 'chunk': '0', 'confidence': '0.5898667'}, []),
 Annotation(chunk, 40, 60, drainage of the fluid, {'entity': 'Symptom', 'sentence': '0', 'chunk': '1', 'confidence': '0.28855002'}, []),
 Annotation(chunk, 75, 86, thoracoscopy, {'entity': 'Procedure', 'sentence': '0', 'chunk': '2', 'confidence': '0.9924'}, []),
 Annotation(chunk, 93, 106, fluid biopsies, {'entity': 'Procedure', 'sentence': '0', 'chunk': '3', 'confidence': '0.76275'}, []),
 Annotation(chunk, 146, 179, epithelioid malignant mesothelioma, {'entity': 'Oncological', 'sentence': '0', 'chunk': '4', 'confidence': '0.53603333'}, []),
 Annotation(chunk, 219, 224, PET CT, {'entity': 'Test', 'sentence': '1', 'chunk': '5', 'confidence': '0.71395004'}, []),
 Annotation(chunk, 240, 255, extensive uptake, {'entity': 'ImagingFindings', 'sentence': '1', 'chunk': '6', 'confidence': '0.5604'}, []),
 Annotation(chunk, 264, 268, chest, {'entity': 'Externa

In [ ]:
print("Clinical Text:\n", text2)

Clinical Text:
 
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007



**Show NER chunks in a pandas dataframe.**

In [ ]:
get_chunk_results(bio_full_result)


 ******************** ner_cellular_biobert Model Results ********************
No Result For This Model

 
 ******************** ner_diseases_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,146,179,epithelioid malignant mesothelioma,Disease,0.95236665
1,1,281,309,pleural pericardial effusions,Disease,0.8773667
2,1,316,330,lymphadenopathy,Disease,0.9807
3,3,456,472,stage III disease,Disease,0.8513667



 
 ******************** ner_events_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,TREATMENT,0.82250005
1,0,40,47,drainage,TREATMENT,0.9375
2,0,52,60,the fluid,PROBLEM,0.92555
3,0,75,86,thoracoscopy,TEST,0.6154
4,0,93,106,fluid biopsies,TEST,0.88645
5,0,137,144,revealed,EVIDENTIAL,0.9861
6,0,146,179,epithelioid malignant mesothelioma,PROBLEM,0.93763334
7,1,217,224,a PET CT,TEST,0.94936675
8,1,233,238,showed,EVIDENTIAL,0.9169
9,1,240,268,extensive uptake in the chest,PROBLEM,0.82536



 
 ******************** ner_bionlp_biobert Model Results ********************


...

,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,DRUG,0.9891667
1,0,40,60,drainage of the fluid,DRUG,0.77637494
2,0,75,86,thoracoscopy,DRUG,0.9375
3,1,240,268,extensive uptake in the chest,ADE,0.90334
4,1,271,309,bilateral pleural pericardial effusions,ADE,0.73015
5,2,347,358,acidic fluid,ADE,0.6747
6,3,456,460,stage,DRUG,0.9395
7,5,600,611,chemotherapy,DRUG,0.9912
8,5,639,647,cisplatin,DRUG,0.992
9,5,693,701,piggyback,DRUG,0.8437



 
 ******************** ner_risk_factors_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,5,616,624,September,PHI,0.9707
1,5,719,727,September,PHI,0.9858
2,5,732,735,2007,I-PHI,0.9828



 
 ******************** ner_living_species_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,1,187,193,patient,HUMAN,0.9994


**Show NER label results in a pandas dataframe.**

In [ ]:
get_token_results(bio_full_result)

,sentence,begin,end,token,ner_ade_biobert,ner_human_phenotype_go_biobert,ner_events_biobert,jsl_ner_wip_greedy_biobert,ner_bionlp_biobert,ner_jsl_biobert,...,ner_deid_biobert,ner_human_phenotype_gene_biobert,ner_clinical_biobert,ner_diseases_biobert,ner_living_species_biobert,ner_anatomy_coarse_biobert,ner_cellular_biobert,ner_anatomy_biobert,ner_deid_enriched_biobert,ner_risk_factors_biobert
0,0,1,2,At,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
1,0,4,7,this,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
2,0,9,12,time,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
3,0,13,13,",",O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
4,0,15,19,chest,B-DRUG,O,B-TREATMENT,B-Procedure,B-Multi-tissue_structure,B-Procedure,...,O,B-HP,B-TREATMENT,O,O,B-Anatomy,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,5,716,717,on,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
124,5,719,727,September,O,O,B-DATE,B-Date,O,B-Date,...,B-DATE,O,O,O,O,O,O,O,B-DATE,B-PHI
125,5,729,729,1,O,O,I-DATE,I-Date,O,I-Date,...,I-DATE,O,O,O,O,O,O,O,I-DATE,O
126,5,730,730,",",O,O,I-DATE,I-Date,O,I-Date,...,I-DATE,O,O,O,O,O,O,O,I-DATE,O
